## Importar librerías

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.pipeline import Pipeline
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, to_onnx, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
from onnxmltools.convert import convert_xgboost as convert_xgboost_booster
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Cargar
tester_files_folder = r"C:\Users\Administrador\AppData\Roaming\MetaQuotes\Terminal\Common\Files"
df_buy = pd.read_csv(os.path.join(tester_files_folder, "buy_training_dataset.csv"))
df_sell = pd.read_csv(os.path.join(tester_files_folder, "sell_training_dataset.csv"))
print(f"Buy -> Trades: {df_buy.shape[0]} Features: {df_buy.shape[1]-1}")
print(f"Sell -> Trades: {df_sell.shape[0]} Features: {df_sell.shape[1]-1}")

Buy -> Trades: 3200 Features: 296
Sell -> Trades: 2768 Features: 296


In [3]:
df_buy

,data_0,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,...,data_287,data_288,data_289,data_290,data_291,data_292,data_293,data_294,data_295,target
0,-0.553430,-0.675676,10.393627,-0.001845,0.001369,81.0,-0.002145,0.001296,92.0,-0.002659,...,64.0,0.000687,0.001628,71.0,-0.000258,0.001635,134.0,0.333333,0.333333,1.0
1,-0.142207,-0.285813,1.626920,-0.000114,0.000391,116.0,-0.000267,0.000396,83.0,-0.000229,...,96.0,-0.000305,0.000538,78.0,-0.000190,0.000539,88.0,0.875000,0.833333,1.0
2,0.193056,0.000830,23.959413,-0.003047,0.002625,927.0,-0.005432,0.002354,919.0,-0.005147,...,234.0,0.004344,0.001271,214.0,0.004344,0.001257,264.0,0.416667,0.333333,1.0
3,-0.734813,0.177390,5.586719,-0.001031,0.001091,304.0,-0.000884,0.001087,260.0,-0.000958,...,436.0,0.000472,0.001878,446.0,0.001106,0.001856,489.0,0.458333,0.500000,1.0
4,-0.264023,-0.747855,9.055182,-0.001086,0.001139,367.0,-0.001655,0.001153,188.0,-0.002049,...,213.0,-0.001839,0.001337,240.0,-0.001226,0.001372,278.0,0.500000,0.666667,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,-0.351430,-0.533829,4.558581,-0.000488,0.000565,192.0,-0.000939,0.000559,203.0,-0.000827,...,121.0,-0.000939,0.000563,184.0,-0.000451,0.000539,146.0,0.916667,0.333333,1.0
3196,1.149069,0.829051,15.574396,-0.000405,0.002454,38.0,-0.003188,0.002497,59.0,-0.004904,...,33.0,0.001014,0.002649,67.0,-0.001873,0.002580,71.0,0.291667,0.833333,0.0
3197,-0.348290,0.364760,2.429500,-0.000236,0.000301,74.0,-0.000472,0.000296,58.0,-0.000539,...,28.0,-0.000303,0.000265,39.0,0.000067,0.000272,14.0,0.333333,0.500000,0.0
3198,-0.647272,-0.214257,5.526539,-0.000128,0.001316,397.0,-0.000906,0.001356,356.0,-0.000765,...,421.0,0.000865,0.001574,412.0,0.000298,0.001635,462.0,0.750000,0.333333,1.0


In [4]:
df_sell

,data_0,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,...,data_287,data_288,data_289,data_290,data_291,data_292,data_293,data_294,data_295,target
0,-0.324988,-0.020805,6.798513,0.000405,0.001211,120.0,0.001091,0.001271,152.0,0.001154,...,260.0,0.002122,0.001459,262.0,0.003497,0.001540,338.0,0.541667,0.500000,0.0
1,0.677953,-0.354522,13.388826,0.002216,0.001299,450.0,0.002130,0.001204,397.0,0.002287,...,474.0,-0.001070,0.001420,463.0,-0.001142,0.001450,497.0,0.750000,0.500000,0.0
2,1.257428,1.454076,7.291260,0.001079,0.000447,133.0,0.001344,0.000379,31.0,0.001112,...,31.0,0.000251,0.000369,48.0,0.000781,0.000364,48.0,0.375000,0.333333,0.0
3,-0.500141,-0.602194,2.409915,0.000126,0.000396,65.0,0.000315,0.000446,41.0,0.000631,...,148.0,0.000883,0.000547,168.0,0.000315,0.000552,210.0,0.750000,0.500000,1.0
4,1.027987,0.754813,9.919270,0.002050,0.001596,1380.0,0.001822,0.001444,399.0,0.003267,...,433.0,-0.002576,0.000866,236.0,-0.001115,0.000740,179.0,0.416667,0.333333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2763,0.250619,-0.852728,14.381383,0.001400,0.001354,256.0,0.002638,0.001279,308.0,0.003050,...,322.0,-0.001152,0.001323,302.0,-0.001398,0.001386,335.0,0.833333,0.166667,0.0
2764,0.166789,-0.871129,3.309161,0.000348,0.000765,184.0,0.000487,0.000785,134.0,0.001113,...,139.0,-0.001321,0.001008,186.0,-0.001529,0.001037,180.0,0.791667,0.500000,0.0
2765,-0.715273,0.339044,5.347409,0.000454,0.001343,565.0,0.001045,0.001309,491.0,0.002501,...,95.0,-0.000318,0.000841,127.0,-0.001044,0.000796,81.0,0.416667,0.833333,0.0
2766,-0.116061,-0.182798,1.774360,0.000200,0.000589,52.0,0.000400,0.000626,81.0,0.000440,...,255.0,-0.000120,0.000944,267.0,-0.000720,0.000964,333.0,0.791667,0.666667,0.0


In [5]:
# Chequear y limpiar
if(df_buy.isna().values.any()):
    df_buy=df_buy.dropna()
if(df_sell.isna().values.any()):
    df_sell=df_sell.dropna()

In [6]:
# Dividir 
# Preparación de los datos de compra
X_buy = df_buy.drop(columns='target').values
y_buy = df_buy['target'].values
X_buy_train = X_buy
y_buy_train = y_buy
# Preparación de los datos de venta
X_sell = df_sell.drop(columns='target').values
y_sell = df_sell['target'].values
X_sell_train = X_sell
y_sell_train = y_sell

In [7]:
# Definir distribuciones para muestreo aleatorio
param_distributions = {
    'xgb__n_estimators': randint(50, 500),
    'xgb__max_depth': randint(3, 9),
    'xgb__learning_rate': uniform(0.01, 0.2),
    'xgb__subsample': uniform(0.6, 1.0),
    'xgb__colsample_bytree': uniform(0.6, 1.0),
    'xgb__gamma': uniform(0, 0.5),
    'xgb__min_child_weight': randint(1, 6),
    'xgb__reg_alpha': uniform(0, 1.0),
    'xgb__reg_lambda': uniform(1.0, 2.0)
}

In [ ]:
# Crear pipe y entrenar
from sklearn.decomposition import PCA
pipe_buy = Pipeline([
    ("scaler", StandardScaler()),
    ('feature_selection', PCA(n_components=0.95)),
    ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
    ])
random_search_buy = RandomizedSearchCV(
    estimator=pipe_buy,
    param_distributions=param_distributions,
    n_iter=100,  # Número de combinaciones a probar
    scoring='accuracy',
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42
)
random_search_buy.fit(X_buy_train, y_buy_train)
model_buy = random_search_buy.best_estimator_
pipe_sell = Pipeline([
    ("scaler", StandardScaler()),
    ('feature_selection', PCA(n_components=0.95)),
    ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
    ])
random_search_sell = RandomizedSearchCV(
    estimator=pipe_sell,
    param_distributions=param_distributions,
    n_iter=100,  # Número de combinaciones a probar
    scoring='accuracy',
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42
)
random_search_sell.fit(X_sell_train, y_sell_train)
model_sell = random_search_sell.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [ ]:
print(f'Model buy score: {random_search_buy.best_score_}')
print(f'Model sell score: {random_search_sell.best_score_}')

In [61]:
update_registered_converter(
    xgb.XGBClassifier,
    "XGBClassifier",
    calculate_linear_classifier_output_shapes,
    convert_xgboost,
    options={'nocl': [False], 'zipmap': [True, False, 'columns']}
)

In [62]:
model_buy_onnx = convert_sklearn(
    model_buy,
    'pipeline_buy_xgboost',
    [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
    target_opset={'': 12, 'ai.onnx.ml': 2}
)
model_sell_onnx = convert_sklearn(
    model_sell,
    'pipeline_sell_xgboost',
    [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
    target_opset={'': 12, 'ai.onnx.ml': 2}
)
mql5_files_folder = "C:/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files"
with open(os.path.join(mql5_files_folder, "model_buy.onnx"), 'wb') as f:
    f.write(model_buy_onnx.SerializeToString())
with open(os.path.join(mql5_files_folder, "model_sell.onnx"), 'wb') as f:
    f.write(model_sell_onnx.SerializeToString())